## **Manipulação Inicial do dados sobre Crise**

In [ ]:
import pandas as pd
import numpy as np

## **Tabela BrStats**

Lendo o arquivo **BrStats** e definindo ';' como separador.

In [ ]:
dados = pd.read_csv('Datasets/BrStats.csv', sep = ';', low_memory= False)
dados

Exibindo todas a linhas e colunas do dataset.

In [ ]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None) 

dados

### Tabela BrStats contendo apenas 'UF', 'Municipio', 'Regiao'e 'PIB'

Para responder a pergunta **9. A partir da análise do PIB, quais municípios se recuperaram da recessão de 2014-2016?** precisaremos apenas dos seguintes dados:
- UF
- Municipio
- Regiao
- PIB
- Ano

Por esse motivo fizemos uma filtragem na tabela BrStats, retirando os dados não necessãrios para essa pergunta.

In [ ]:
dados_filtrados = dados[['UF', 'Municipio', 'Regiao', 'PIB', 'Ano']]

dados_filtrados

É possível salvar a nova tabela em um arquivo .csv separado:

In [ ]:
dados_filtrados.to_csv("Datasets/BrStats_crise.csv", index=False)

## **Tabela sidra**

Como mencionado no README, fizemos uma busca no Sidra para encontrar dados do PIB dos municípios brasileiros dos anos de 2013 (anterior a crise) até 2016 (fim da crise), o arquivo gerado é o sidra.csv.

Abaixo estamos carregando e manipulando o arquivo sidra, para que possamos unir as duas tabelas de dados.

In [ ]:
# Carregando o arquivo CSV com a codificação correta UTF-8, pois estava perdendo acentos
dados_sidra = pd.read_csv('Datasets/sidra.csv', skiprows=1, encoding='utf-8')

# Renomeando as colunas
dados_sidra.columns = ['Município', '2013', '2014', '2015', '2016', '2017']

# Removendo linhas sem dados úteis que vieram do PDF (Fonte, Notas, etc.)
dados_sidra = dados_sidra.dropna() 
dados_sidra = dados_sidra[~dados_sidra['Município'].str.contains('Fonte:|Notas|1 -', na=False)] 

# Salvando o arquivo corrigido com codificação UTF-8
dados_sidra.to_csv('Datasets/sidra_limpos.csv', index=False, encoding='utf-8-sig')

Um detalhe a ser corrigido antes de unirmos as tabelas do BrStats com a do Sidra é a maneira como municípios e UF estão registrados, enquanto o dataset BrStats possui uma coluna chamada 'UF', o dataset do Sidra escreve a UF na mesma coluna do município, e por isso devemos criar uma nova coluna nesse dataset.

In [ ]:
# Criando a coluna UF
if 'UF' not in dados_sidra.columns:
    dados_sidra['UF'] = ''

# Separaando Município e UF
dados_sidra[['Município', 'UF']] = dados_sidra['Município'].str.extract(r'(.+) \((\w{2})\)')

dados_sidra

## **Unindo os dois Datasets**

O dataset BrStats não possui colunas específicas para os anos em que os PIBs foram registrados, então optamos por fazer divisão para facilitar a análise.

In [ ]:
# Criando um DataFrame onde cada ano vira uma coluna
dados = dados.pivot_table(
    index=['UF', 'Municipio', 'Regiao'],  # Colunas que permanecerão como índice
    columns='Ano',                        # Coluna que será transformada em novas colunas
    values='PIB',                         # Valores que preencherão as novas colunas
    aggfunc='first'                       # Função de agregação (usamos 'first' pois não há duplicatas)
)

# Resetar o índice para trazer UF, Municipio e Regiao de volta como colunas
dados = dados.reset_index()

dados

Outro problema a ser corrigido é a coluna 'Municipio' no dataset do Sidra, que possui acentuação, enquanto a do BrStats não possui, e precisaremos que elas possuam nomeclatura idêntica para uni-las.

In [ ]:
dados_sidra = dados_sidra.rename(columns={'Município': 'Municipio'})

Unindo os dois datasets utilizando como base as colunas 'Municipio' e 'UF'.

In [ ]:
dados_crise = dados.merge(dados_sidra, on=['Municipio', 'UF'])

# Salvar o arquivo corrigido com codificação UTF-8
dados_crise.to_csv('Datasets/dados_crise.csv', index=False, encoding='utf-8-sig')

dados_crise

É possível perceber que o dataset ainda precisa de alguns ajustes, como ordenar corretamente os anos e remover os que se repetem.

In [ ]:
dados_crise = pd.read_csv('Datasets/dados_crise.csv')
dados_crise

In [ ]:
# Removendo colunas duplicadas
cols_to_remove = ['2016.1', '2017.1']
dados_crise = dados_crise.drop(columns=[col for col in cols_to_remove if col in dados_crise.columns])

# Reordenando colunas
fixed_cols = ['UF', 'Municipio', 'Regiao']
year_cols = sorted([col for col in dados_crise.columns if col.isdigit()])
dados_crise = dados_crise[fixed_cols + year_cols]

# Salvando o resultado
output_file = 'Datasets/dados_final.csv'
dados_crise.to_csv(output_file, index=False)

dados_crise

Com isso obtivemos um dataset com apenas os dados necessários para a análise do PIB antes, durante e depois da crise de 2014-2016.

# **📉 10. Como a taxa de desemprego e o PIB foram afetadas durante a pandemia**

In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset_original = pd.read_csv('https://raw.githubusercontent.com/anaC-Fernandes/ProjetoDados/refs/heads/main/BrStats.csv', sep = ";", low_memory=False)
df = dataset_original

In [3]:
# Tratamento da formatação dos dados
for coluna in df.columns:
    df[coluna] = df[coluna].astype(str)  # Garante que é string
    df[coluna] = df[coluna].str.replace('.', '', regex=False)  # Remove separador de milhar
    df[coluna] = df[coluna].str.replace(',', '.', regex=False)  # Converte decimal
    df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
df

for coluna in df.columns:
  df[coluna] = df[coluna].astype(str)  # Garante que é string
  df[coluna] = df[coluna].str.replace('nan', '0', regex=False)  # Converte decimal

df

# Tratamento de tipo

df['Ano'] = df['Ano'].astype(int)
df['CDMunicipio'] = df['CDMunicipio'].astype(str)
df['Populacao'] = df['Populacao'].astype(int)
df['PessoalOcupado'] = df['PessoalOcupado'].astype(int)
df['PessoalAssalariado'] = df['PessoalAssalariado'].astype(int)
df['VrSalarios'] = df['VrSalarios'].astype(int)
df['PIB'] = df['PIB'].astype(int)
df['QtEmpresas'] = df['QtEmpresas'].astype(int)
df['AreaPlantada_h'] = df['AreaPlantada_h'].astype(int)
df['AreaColhida_h'] = df['AreaColhida_h'].astype(int)
df['VlProducaoAgricola'] = df['VlProducaoAgricola'].astype(int)
df['VlProducaoPecuaria'] = df['VlProducaoPecuaria'].astype(int)
df['Area'] = df['Area'].astype(float)
df['Importacoes_US$'] = df['Importacoes_US$'].astype(float)
df['Exportacoes_US$'] = df['Exportacoes_US$'].astype(float)
df['Receitas_R$'] = df['Receitas_R$'].astype(float)
df['Transferencias_correntes_R$'] = df['Transferencias_correntes_R$'].astype(float)
df['Transferencias_capital_R$'] = df['Transferencias_capital_R$'].astype(float)
df['NrNascimentos'] = df['NrNascimentos'].astype(int)
df['NrObitosInfantis'] = df['NrObitosInfantis'].astype(int)
df['povoamento'] = df['povoamento'].astype(float)



/tmp/ipykernel_61714/374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
/tmp/ipykernel_61714/374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
/tmp/ipykernel_61714/374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[coluna] = pd.to_numeric(df[coluna], errors='ignore')  # Converte para número, ignorando erros
/tmp/ipykernel_61714/374510504.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future 

In [4]:
#tabela usada pra analisar os anos da Covid
tabela = df
tabela = tabela.drop(columns='AreaPlantada_h')
tabela = tabela.drop(columns='AreaColhida_h')
tabela = tabela.drop(columns='NrNascimentos')
tabela = tabela.drop(columns='NrObitosInfantis')
tabela = tabela.drop(columns='Area')
tabela = tabela.drop(columns='Importacoes_US$')
tabela = tabela.drop(columns='Exportacoes_US$')
tabela = tabela.drop(columns='Receitas_R$')
tabela = tabela.drop(columns='Transferencias_correntes_R$')
tabela = tabela.drop(columns='Transferencias_capital_R$')
tabela = tabela.drop(columns='VlProducaoAgricola')
tabela = tabela.drop(columns='VlProducaoPecuaria')

In [5]:
tabela['PIB_per_capita'] = tabela['PIB'] / tabela['Populacao'].replace(0, pd.NA)
tabela.sort_values(by='PIB_per_capita', ascending=False)


,Ano,CDMunicipio,Populacao,PessoalOcupado,PessoalAssalariado,VrSalarios,PIB,QtEmpresas,povoamento,UF,Municipio,Regiao,PIB_per_capita
10625,2020,1502152,38103,8553,7407,228990,22522725,9830,12.110,PA,Canaã dos Carajás,Norte,591.101094
15081,2018,3204302,11488,2730,2515,67664,6693537,1650,19.587,ES,Presidente Kennedy,Sudeste,582.654683
23853,2019,3204302,11574,2930,2669,70037,5383038,2000,19.734,ES,Presidente Kennedy,Sudeste,465.09746
8363,2019,3520400,34970,10333,8901,306489,14965490,1272,100.633,SP,Ilhabela,Sudeste,427.952245
6172,2018,3520400,34333,9378,8243,271697,14400916,9810,98.800,SP,Ilhabela,Sudeste,419.448228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32419,2017,4212650,0,0,0,0,0,0,0.000,SC,Pescaria Brava,Sul,<NA>
32537,2020,1504752,0,0,0,0,0,0,0.000,PA,Mojui dos Campos,Norte,<NA>
32691,2017,4314548,0,0,0,0,0,0,0.000,RS,Pinto Bandeira,Sul,<NA>
32723,2016,4220000,0,0,0,0,0,0,0.000,SC,Balneario Rincao,Sul,<NA>
